# INITIALISATION


**Importation des librairies**

In [122]:
import spacy
from spacy import displacy
from neo4j import GraphDatabase
import glob, os
from nltk import tokenize

**Initatilisation des variables**

In [124]:
nbRelationAnn = 0
nbEntiteAnn = 0

nbRelationPred = 0
nbEntitePred = 0

In [125]:
keys = {
    'Person': 'PER',
    'Location': 'LOC',
    'City': 'LOC',
    'LocalRegion': 'LOC',
    'Country': 'LOC',
    'WorldRegion': 'LOC',
    'Organization': 'ORG',
    'Association': 'ORG',
    'GeopoliticalEntity': 'ORG',
    'Company': 'ORG',
    'Media': 'ORG',
    'Role': 'ROL',
    'Currency': 'CUR'
}

**Fonction donnant le contexte des relations**

In [123]:
def getContext(tabText,nbCaractere):
    lenTxt1 = 0
    lenTxt2 = len(tabText[0])
    for i in range(0,len(tabText)):
        if nbCaractere > lenTxt1 and nbCaractere <= lenTxt2 :
            return (i)
        else :
            lenTxt1 = lenTxt1 + len(tabText[i])
            if( i +1 < len(tabText) ) :
                lenTxt2 = lenTxt2 + len(tabText[i+1])


# IMPORTATION DANS NEO4J DES ENTITES ET DES RELATIONS A PARTIR DES ANNOTATIONS

**Exemple d'utilisation avec le sample M**

In [126]:
#Initialisation de la connection a Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "i-0578b6cebb222aa18"))

#Chemin du dossier contenant les fichiers
path = "Sample_M"
files = [f for f in glob.glob(path +"**/*.ann" )]
for file_path in files:
    file = open(file_path, "r",encoding='utf-8-sig')
    line = file.readline()
    print(file_path)
    fileTxt = open(file_path.replace("ann","txt"), "r")
    text= fileTxt.read()
    tokText=tokenize.sent_tokenize(text, language='french')
    

    #Dictionnaire des entité et relation du fichier
    tabLine = {}

    #Les entités où l'on souhaite avoir des doublons
    listeDoublon = ["Role","Activity",'ROL']

    #Lecture du fichier ligne par ligne
    while line:
        #driver = GraphDatabase.driver("bolt://15.188.27.124:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
        driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
        strLine = line.strip()
        
        #traitement de la ligne en séparant les différents champs
        strLine = strLine.replace("\t"," ")
        strLine = strLine.replace("\uffef","")
        indice = strLine.split(" ",4)[0]

        #Traitement dans le cas d'une entité
        if strLine[0] == 'T' :

            nbEntiteAnn += 1
            
            tabLine[indice] = strLine.split(" ",4)
            
            tabLine[indice][0] = tabLine[indice][0].replace('"',"'" )
            tabLine[indice][4] = tabLine[indice][4].replace('"',"'" )
            
            nomT = tabLine[indice][1]
            if keys.get(nomT) != None:
                nomT = keys.get(nomT)

            #Requete pour ajouter l'entité à Neo4j
            if nomT in listeDoublon :
                query = "CREATE (:" + nomT + "{nom:\"" + tabLine[indice][4] + "\" , id :\"" + tabLine[indice][0] + "\"}) "  
            else :
                query = "MERGE (:" + nomT + "{nom:\"" + tabLine[indice][4] + "\"}) "

        #traitement dans le cas d'une relation
        elif strLine[0] == 'R' :

            try :
                
                nbRelationAnn += 1
                
                tabLine[indice] = strLine.split(" ",4)
                tabLine[indice][2] = tabLine[indice][2].split(":")[1]
                tabLine[indice][3] = tabLine[indice][3].split(":")[1]
                nomR = tabLine[indice][1]
                R1 = tabLine[indice][2]
                R2 = tabLine[indice][3]

                tabLine[R1][1] = tabLine[R1][1].replace('"',"'" )
                tabLine[R1][4] = tabLine[R1][4].replace('"',"'" )

                tabLine[R2][1] = tabLine[R2][1].replace('"',"'" )
                tabLine[R2][4] = tabLine[R2][4].replace('"',"'" )


                if keys.get(tabLine[R1][1]) != None:
                    tabLine[R1][1] = keys.get(tabLine[R1][1])
                    
                if keys.get(tabLine[R2][1]) != None:
                    tabLine[R2][1] = keys.get(tabLine[R2][1])
                    
                nomR1 = tabLine[R1][1]
                nomR2 = tabLine[R2][1]

                debutRole = int(tabLine[R1][2])
                finRole = int(tabLine[R2][3])

                nb1=getContext(tokText,debutRole)
                nb2=getContext(tokText,finRole)
                if (nb1 != nb2 ):
                    context = tokText[nb1]+" | "+tokText[nb2]
                else : 
                    context = tokText[nb1]
                
                context= context.replace('"',"'")
                #Les différents cas de figure si l'on a affaire à des chaines de relations avec des doublons
                if nomR1 in listeDoublon and nomR2 not in listeDoublon :
                    query =  "MATCH ( T1:" + tabLine[R1][1] + "{nom:\"" + tabLine[R1][4] + "\" , id :\"" + R1 + "\"} ) "
                    query += "MATCH ( T2:" + tabLine[R2][1] + "{nom:\"" + tabLine[R2][4] + "\"} ) "
            
                    query += "OPTIONAL MATCH (( T1 )-[r1:" + nomR + "]->( T2 )) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + nomR + " {context : [\""+context+"\"],type:'A',file:\""+file_path+"\"} ]->( T2 ) ) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "

                elif nomR1 not in listeDoublon and nomR2 in listeDoublon:
                    query =  "MATCH ( T1:" + tabLine[R1][1] + "{nom:\"" + tabLine[R1][4] + "\"} ) "
                    query += "MATCH ( T2:" + tabLine[R2][1] + "{nom:\"" + tabLine[R2][4] + "\" , id :\"" + R2 + "\"} ) "
                    query += "OPTIONAL MATCH (( T1 )-[r1:" + nomR + "]->( T2 )) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + nomR + " {context : [\""+context+"\"],type:'A',file:\""+file_path+"\"} ]->( T2 ) ) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "

                elif nomR1 in listeDoublon and nomR2 in listeDoublon :
                    query =  "MATCH ( T1:" + tabLine[R1][1] + "{nom:\"" + tabLine[R1][4] + "\" , id :\"" + R1 + "\"} ) "
                    query += "MATCH ( T2:" + tabLine[R2][1] + "{nom:\"" + tabLine[R2][4] + "\" , id :\"" + R2 + "\"} ) "

                    query += "OPTIONAL MATCH (( T1 )-[r1:" + nomR + "]->( T2 )) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + nomR + " {context : [\""+context+"\"],type:'A',file:\""+file_path+"\"} ]->( T2 ) ) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "

                else :
                    query =  "MATCH ( T1:" + tabLine[R1][1] + "{nom:\"" + tabLine[R1][4] + "\"} ) "
                    query += "MATCH ( T2:" + tabLine[R2][1] + "{nom:\"" + tabLine[R2][4] + "\"} ) "
            

                    query += "OPTIONAL MATCH (( T1 )-[r1:" + nomR + "]->( T2 )) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + nomR + " {context : [\""+context+"\"],type:'A',file:\""+file_path+"\"} ]->( T2 ) ) "
                    query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + nomR + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "
                
            except KeyError as e:
                print("--------------------------------")
                print(file)
                print(e)
                print(strLine)
            except TypeError as e2:
                pass

        #On lit la ligne suivante
        line = file.readline()
        #On execute la requete
        driver.session().run(query) 
    #traitement finale où l'on supprime tous les doublons
    query= """
    MATCH (a)-[r1]->(b) , (c)-[r2]->(d)
    WHERE 
    b<>d and
    a.nom = c.nom and b.nom = d.nom
    call apoc.refactor.mergeNodes([b,d],{properties:"combine", mergeRels:true}) yield node
    return "none"
    """
    driver.session().run(query) 
    #requete permettant de supprimer la propriété id qui été utilisé pour les relations
    query= """
    MATCH (n)
    remove n.id
    return n
    """
    driver.session().run(query) 

print("done")

Sample_M/M_ACCOR.ann
Sample_M/M_ACQUISITION.ann
Sample_M/M_ACTIVITE.ann
Sample_M/M_ADMINISTRATION.ann
Sample_M/M_CONTINENT.ann
Sample_M/M_DOLLAR.ann
Sample_M/M_FRANCE.ann
Sample_M/M_FUSION.ann
Sample_M/M_IMMOBILIER.ann
Sample_M/M_INFORMATIQUE.ann
done


# PREDICTION

**Chargement du model**

In [127]:
nlp = spacy.load("NER_Model_11Dec")

**Importation dans Neo4j des entités prédites**

In [129]:
#Initialisation de la connection a Neo4j
#driver = GraphDatabase.driver("bolt://15.188.27.124:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
#Chemin du dossier contenant les fichiers
path = "Sample_M"
files = [f for f in glob.glob(path +"**/*.txt" )]
listeDoublon = ["ROL","Role","Activity"]

for file_path in files:
    file = open(file_path, "r",encoding='utf-8-sig')
    print(file_path)
    
    text= file.read()
    
    doc = nlp(text)
    for token in doc.ents:
        #if token.label_  in listeDoublon :
        nbEntitePred += 1
        if token.label_  in listeDoublon :
            
            #query = "MERGE (:" + token.label_ + "{nom:\"" + '{}'.format(token.start_char) + "\" , end\"" + '{}'.format(token.end_char) + "\" }) "
            query =  "CREATE (:" + token.label_ + "{nom:\"" + token.text + "\" , id:\"" + '{}'.format(token.start_char) + "\" }) "
            #idEnt+=1
        else :
            query = "MERGE (:" + token.label_ + "{nom:\"" + token.text + "\"}) "
        
            #print(token.text, token.start_char, token.end_char, token.label_)
            #print(query)
        driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
        driver.session().run(query)  
print("done")

Sample_M/M_ACCOR.txt
Sample_M/M_ACQUISITION.txt
Sample_M/M_ACTIVITE.txt
Sample_M/M_ADMINISTRATION.txt
Sample_M/M_CONTINENT.txt
Sample_M/M_DOLLAR.txt
Sample_M/M_FRANCE.txt
Sample_M/M_FUSION.txt
Sample_M/M_IMMOBILIER.txt
Sample_M/M_INFORMATIQUE.txt
done


**Fonction pour extraire les relations hasRole**

In [130]:
def relate(TEXTS):
    relationsTab =[]
    for text in TEXTS:
        print('\n>> '+text+'\n')
        doc = nlp(text)
        relations = extract_relations(doc)
        relations.append(text)
        relationsTab.append(relations)

    return relationsTab

def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result


def extract_relations(doc):
    
    spans = list(doc.ents)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    for target in filter(lambda w: w.ent_type_ == 'ROL', doc):
        if target.dep_ == "obl":
            subject = [w for w in target.head.lefts if w.dep_ == "nsubj:pass"]
            if subject:
                if subject[0].ent_type_ == 'PER':
                    relations.append(('hasRole','PER',str(subject[0]),'ROL',str(target),target.idx))
                if subject[0].ent_type_ == 'ORG':

                    relations.append(('roleDepartment','ROL', str(target),'ORG', str(subject[0]),target.idx))
            #'Ce matin, Valérie Pécresse sera installée à la présidence de la région Ile-de-France.',
            #('hasRole','Valérie Pécresse','présidence')

        if target.dep_ == "appos":
            if target.head.ent_type_ == 'PER':
                relations.extend([('hasRole','PER',str(target.head),'ROL', str(target),target.idx)])
            if target.head.ent_type_ == 'ORG':

                relations.extend([('roleDepartment','ROL', str(target),'ORG' ,str(target.head),target.idx)])
            #« Ce sont des entreprises très souples, dans lesquelles la direction peut prendre très vite des décisions », 
            #commente Jacky Lintignat, directeur général de KPMG France.',
            #('hasRole', 'Jacky Lintignat', 'directeur général')
            else :
                mod = [target.head]
                if mod:
                    mod = [w for w in mod[0].children if w.dep_ == "nmod"]
                    if mod:
                        mod = mod[0]
                        if mod.ent_type_ == 'PER':

                            relations.extend([('hasRole','PER',str(mod),'ROL', str(target),target.idx)])
                        if mod.ent_type_ == 'ORG':
                            relations.extend([('roleDepartment','ROL',str(target),'ORG', str(mod),target.idx)])
                        #'« Ce discours était un signal politique important, analyse Shahin Vallée, chercheur invité au cercle 
                        #de réflexion européen Bruegel.',
                        #('hasRole', 'Shahin Vallée', 'chercheur')
            mod1 = [w for w in target.children if w.dep_ =='nmod' and w.ent_type_ in ["PER","ORG"]]
            if mod1:
                mod1 = mod1[0]
                if mod1.ent_type_ == 'PER':
                    relations.extend([('hasRole','PER',str(mod1),'ROL', str(target),target.idx)])
                if mod1.ent_type_ == 'ORG':
                    relations.extend([('roleDepartment','ROL', str(target),'ORG', str(mod1),target.idx)])
                #"Après Hubert Joly, le patron de l'américain Best Buy, c'est Alain Caparros, celui de l'allemand Rewe, qui a 
                #officiellement décliné.",
                #('roleDepartment', 'patron', 'Best Buy')
            
        if target.dep_ == "acl":
            mod1 = [w for w in target.children if w.dep_ =='nmod' and w.ent_type_ in ["PER","ORG"]]
            if mod1:
                mod1 = mod1[0]
                if mod1.ent_type_ == 'PER':
                    relations.extend([('hasRole','PER',str(mod1),'ROL',  str(target),target.idx)])
                if mod1.ent_type_ == 'ORG':
                    relations.extend([('roleDepartment','ROL', str(target),'ORG',  str(mod1),target.idx)])
                #"« Si la concurrence s'intensifie, ils ne pourront pas y échapper », prédit Philippe Lerouge, fondateur 
                #du Salon Mobile Payment.",
                #('roleDepartment', 'fondateur', 'Salon Mobile Payment')
        if target.dep_ == "nsubj":
            mod = [w for w in target.children if w.dep_ in ["obj","appos"]]
            if mod:
                mod = mod[0]
                if mod.ent_type_ == 'PER':
                    relations.extend([('hasRole','PER',str(mod),'ROL',  str(target),target.idx)])
                if mod.ent_type_ == 'ORG':
                    relations.extend([('roleDepartment','ROL', str(target),'ORG',  str(mod),target.idx)])
                #'Le PDG de la division aviation commerciale d\'Airbus, Fabrice Brégier, a estimé que cette livraison marquait "
                #la renaissance de l\'aviation d\'Iran Air" et constituait "un des développements les plus importants de 
                #l\'industrie (aéronautique) depuis bien des années".',
                #('hasRole', 'Fabrice Brégier', 'PDG')
        
        subject = [w for w in target.children if w.dep_ in ("nmod", "nsubj","obl")]
        if subject:
            if subject[0].ent_type_ == 'PER':
                relations.extend([('hasRole','PER',str(s),'ROL',  str(target),target.idx) for s in subject])
            if subject[0].ent_type_ == 'ORG':
                relations.extend([('roleDepartment','ROL', str(target),'ORG',  str(s),target.idx) for s in subject])
            #'« Les mauvais taux de transformation ne nous incitent pas à investir pour adapter nos jeux aux terminaux sous 
            #Android », explique Gonzague de Vallois, vice-président de Gameloft.',
            #('roleDepartment', 'vice-président', 'Gameloft')]})

    return relations

In [131]:
def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result

def extract_relationsCurrency(doc):
    
    spans = list(doc.ents)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)
            
    relations = []
    nbCurrency = 0
    nbCurrencyPred = 0
    for target in filter(lambda w: w.ent_type_ == 'CUR', doc) :
        bol = nbCurrencyPred
        for i in range(-2,2):
            if ( doc[target.i+i].ent_type_ == 'MISC' or doc[target.i+i].dep_ == 'nummod'):
                relations.extend([('hasCurrency','MISC',str(doc[target.i+i].text),'CUR',  str(target),target.idx)])

    return relations

In [132]:
#Initialisation de la connection a Neo4j

#Chemin du dossier contenant les fichiers
path = "Sample_M"
files = [f for f in glob.glob(path +"**/*.txt" )]
listeDoublon = ["ROL","Role","Activity"]
nbRel = 0
#relations=[]
for file_path in files:
    file = open(file_path, "r",encoding='utf-8-sig')
    print(file_path)
    
    text= file.read()
    tokText=tokenize.sent_tokenize(text, language='french')
    relations=[]
    doc = nlp(text)
    #relations.extend(extract_relations(doc))
    relations.extend(extract_relationsCurrency(doc))

Sample_M/M_ACCOR.txt
Sample_M/M_ACQUISITION.txt
Sample_M/M_ACTIVITE.txt
Sample_M/M_ADMINISTRATION.txt
Sample_M/M_CONTINENT.txt
Sample_M/M_DOLLAR.txt
Sample_M/M_FRANCE.txt
Sample_M/M_FUSION.txt
Sample_M/M_IMMOBILIER.txt
Sample_M/M_INFORMATIQUE.txt


In [133]:
#Initialisation de la connection a Neo4j
#driver = GraphDatabase.driver("bolt://15.188.27.124:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
#Chemin du dossier contenant les fichiers
path = "Sample_M"
files = [f for f in glob.glob(path +"**/*.txt" )]
listeDoublon = ["ROL","Role","Activity","CUR","MISC"]
nbRel = 0

for file_path in files:
    file = open(file_path, "r",encoding='utf-8-sig')
    print(file_path)
    
    text= file.read()
    tokText=tokenize.sent_tokenize(text, language='french')
    
    doc = nlp(text)
    relations=[]
    relations.extend(extract_relations(doc))
    relations.extend(extract_relationsCurrency(doc))

    for rel in relations[:len(relations)-1]:
        nbRel+=1
        nbRelationPred += 1
        
        query = ""
        context=''
        nbContext = getContext(tokText,rel[5])
        context = tokText[nbContext].replace('"',"'" )
        
        
        
        if rel[1] in listeDoublon and rel[3] not in listeDoublon :
            
            query =  "MATCH ( T1:" + rel[1] + "{nom:\"" + rel[2] + "\" , id :\"" + '{}'.format(rel[5]) + "\"} ) "
            query += "MATCH ( T2:" + rel[3] + "{nom:\"" + rel[4] + "\"} ) "

            query += "OPTIONAL MATCH (( T1 )-[r1:" + rel[0] + "]->( T2 )) "
            query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + rel[0] + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + rel[0] + " {context : [\""+context+"\"],type : 'P',file:\""+file_path+"\"} ]->( T2 ) ) "
            query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + rel[0] + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "

        elif rel[1] not in listeDoublon and rel[3]  in listeDoublon:
            query =  "MATCH ( T1:" + rel[1] + "{nom:\"" + rel[2] + "\"} ) "
            query += "MATCH ( T2:" + rel[3] + "{nom:\"" + rel[4] + "\" , id :\"" + '{}'.format(rel[5]) + "\"} ) "
            
            query += "OPTIONAL MATCH (( T1 )-[r1:" + rel[0] + "]->( T2 )) "
            query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + rel[0] + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + rel[0] + " {context : [\""+context+"\"],type : 'P',file:\""+file_path+"\"} ]->( T2 ) ) "
            query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + rel[0] + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "

        elif rel[1] in listeDoublon and rel[3]  in listeDoublon :
            
            query =  "MERGE ( T1:" + rel[1] + "{nom:\"" + rel[2] + "\" , id :\"" + '{}'.format(rel[5]) + "\"} ) -[:" + rel[0] + " {context : [\""+context+"\"],type : 'P',file:\""+file_path+"\"} ]->  ( T2:" + rel[3] + "{nom:\"" + rel[4] + "\" , id :\"" + '{}'.format(rel[5]) + "\"} ) "

        else :
            query =  "MATCH ( T1:" + rel[1] + "{nom:\"" + rel[2] + "\"} ) "
            query += "MATCH ( T2:" + rel[3] + "{nom:\"" + rel[4] + "\"} ) "


            query += "OPTIONAL MATCH (( T1 )-[r1:" + rel[0] + "]->( T2 )) "
            query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + rel[0] + "]->( T2 )) = False  THEN [1] ELSE [] END | MERGE ( T1 )-[:" + rel[0] + " {context : [\""+context+"\"],type : 'P',file:\""+file_path+"\"} ]->( T2 ) ) "
            query +="FOREACH (usertype IN CASE WHEN exists (( T1 )-[:" + rel[0] + "]->( T2 )) = True  THEN [1] ELSE [] END | SET r1.context = r1.context + \""+ context +"\") "

   
        driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "i-0578b6cebb222aa18"))
        driver.session().run(query) 
print("nb relations : {}".format(nbRel))

Sample_M/M_ACCOR.txt
Sample_M/M_ACQUISITION.txt
Sample_M/M_ACTIVITE.txt
Sample_M/M_ADMINISTRATION.txt
Sample_M/M_CONTINENT.txt
Sample_M/M_DOLLAR.txt
Sample_M/M_FRANCE.txt
Sample_M/M_FUSION.txt
Sample_M/M_IMMOBILIER.txt
Sample_M/M_INFORMATIQUE.txt
nb relations : 67


In [134]:
print("Entites annoté : {} | entites predites {}".format(nbEntiteAnn,nbEntitePred))
print("Relations annoté : {} | relations predites {}".format(nbRelationAnn,nbRelationPred))

Entites annoté : 668 | entites predites 583
Relations annoté : 233 | relations predites 67
